In [1]:
# Importing libraries needed to scrape
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
# This will be heavily commented as you can imagine it's easy to forget a python library lol

In [2]:
needed_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

res = requests.get("https://www.themoviedb.org/movie", headers=needed_headers)
res.status_code
# An exit status of 200 means OK :)


200

In [3]:
# Creating an instance of the BeautifulSoup class
# also im going to be using lxml instead of the conventional html.parser lets see how that goes
soup = BeautifulSoup(res.text, "lxml")

# We won't call soup for our own sanity!

In [4]:
web_content = soup.get_text().strip()
# using .strip() right now gets rid of trailing whitespaces but not leading??? wtf? will figure out
print(web_content[:15])

Popular Movies 


### Extracting the title of the web page using various methods:

In [5]:
# Extracting title
soup.title


<title>Popular Movies — The Movie Database (TMDB)</title>

In [6]:
# Another method of fetching the title
soup.select("title")[0].getText()

'Popular Movies — The Movie Database (TMDB)'

### We will use functions to help ease our workflow:

In [7]:
# Generalizing initial task with the help of functional stuff
def vaccum(url: str) -> BeautifulSoup:
    try:
        # I am not a robot lmao
        needed_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        res_func = requests.get(url,headers=needed_headers)

        # Raise exception if status code is 5xx or 4xx
        res_func.raise_for_status()

        return BeautifulSoup(res_func.text, "lxml")
    
    except requests.exceptions.RequestException as e:
        # Catch all request-related errors 
        print(f"An Error occured!: {e}")
        return None
    
    finally:
        print("Excecution FIN")

In [8]:
# test case 1: a working url
vaccumed = vaccum("https://www.themoviedb.org/movie")
# This works, won't call it for obvious reasons haha

Excecution FIN


In [9]:
# test case 2: malformed/incorrect URLs
vaccum("https://www.themovipepepe.org/what")

An Error occured!: HTTPSConnectionPool(host='www.themovipepepe.org', port=443): Max retries exceeded with url: /what (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017DE9E24620>: Failed to resolve 'www.themovipepepe.org' ([Errno 11001] getaddrinfo failed)"))
Excecution FIN


In [10]:
# Displaying the name of the first movie
first_movie = soup.select(".content h2")[3].getText()
first_movie

'Sonic the Hedgehog 3'

In [11]:
# Extracting User rating of the first movie
span_class = soup.select_one(".user_score_chart .icon")['class'][1]
span_class
perc1 = span_class.split("icon-r")[1]
print(f"User score of the movie: {first_movie} is {perc1}%")




User score of the movie: Sonic the Hedgehog 3 is 79%


In [12]:
# we're gonna use regex for extracting the part of the url following the string “https://www.themoviedb.org/” 
text = str(soup.select(".content h2"))
match = re.search(r'movie/\d\d\d\d\d\d',text)
match.group()
# This is going to help us later (i think)

'movie/939243'

In [13]:
# Grabbing the title of all the movies
def title_grabber(soup: BeautifulSoup) -> list:
    titles = []
    movies = soup.select(".content h2")[3:]
    for movie in movies:
        a_tag = movie.find("a", title = True)
        if a_tag:
            titles.append(a_tag["title"])
    return titles
    
        
title_grabber(vaccumed)

['Sonic the Hedgehog 3',
 'Moana 2',
 'Kraven the Hunter',
 'बैक इन ऐक्शन',
 'Alarum',
 'Nosferatu',
 'Mufasa: The Lion King',
 'The Gardener',
 'Devara: Part 1',
 'Gladiator II',
 'Venom: The Last Dance',
 'Werewolves',
 'Kingdom IV: Return of the Great General',
 'Sniper: The Last Stand',
 'Number 24',
 'The Substance',
 'Red One',
 'Le clitoris',
 'Aftermath',
 'Wicked']

In [25]:
# Grabbing the user ratings of all the movies in a page
def rating_grabber(soup: BeautifulSoup) -> list:
    user_score = []
    scores = soup.select(".content .user_score_chart")

    for score in scores:
        # edge case
        if score is None:
            user_score.append("No rating")
        else:
            percent = score["data-percent"]
            user_score.append(percent)
        
        

    return user_score
rating_grabber(vaccumed)

['79',
 '72',
 '66',
 '67',
 '57',
 '66',
 '75',
 '61',
 '70',
 '68',
 '68',
 '62',
 '73',
 '70',
 '73',
 '71',
 '71',
 '64',
 '63',
 '69']

In [15]:
soup.select(".content h2")

[<h2>Sort</h2>,
 <h2>Where To Watch <span>71</span></h2>,
 <h2>Filters</h2>,
 <h2><a href="/movie/939243-sonic-the-hedgehog-3" title="Sonic the Hedgehog 3">Sonic the Hedgehog 3</a></h2>,
 <h2><a href="/movie/1241982-moana-2" title="Moana 2">Moana 2</a></h2>,
 <h2><a href="/movie/539972-kraven-the-hunter" title="Kraven the Hunter">Kraven the Hunter</a></h2>,
 <h2><a href="/movie/993710-back-in-action" title="बैक इन ऐक्शन">बैक इन ऐक्शन</a></h2>,
 <h2><a href="/movie/1249289-alarum" title="Alarum">Alarum</a></h2>,
 <h2><a href="/movie/426063-nosferatu" title="Nosferatu">Nosferatu</a></h2>,
 <h2><a href="/movie/762509-mufasa-the-lion-king" title="Mufasa: The Lion King">Mufasa: The Lion King</a></h2>,
 <h2><a href="/movie/1255788-le-jardinier" title="The Gardener">The Gardener</a></h2>,
 <h2><a href="/movie/811941-part-1" title="Devara: Part 1">Devara: Part 1</a></h2>,
 <h2><a href="/movie/558449-gladiator-ii" title="Gladiator II">Gladiator II</a></h2>,
 <h2><a href="/movie/912649-venom-the

In [16]:
# Extracting the HTML golddd of all the individual pages of movies
def html_content(soup: BeautifulSoup) -> list:
    my_content = str(soup.select(".content h2"))
    my_list_of_contents = []

    stuff = re.findall(r"movie/\d\d\d\d\d\d", my_content)
    my_list_of_contents.extend(stuff)

    return my_list_of_contents
   
html_content(vaccumed)
    

['movie/939243',
 'movie/124198',
 'movie/539972',
 'movie/993710',
 'movie/124928',
 'movie/426063',
 'movie/762509',
 'movie/125578',
 'movie/811941',
 'movie/558449',
 'movie/912649',
 'movie/970450',
 'movie/124132',
 'movie/141008',
 'movie/122224',
 'movie/933260',
 'movie/845781',
 'movie/426889',
 'movie/108101',
 'movie/402431']

In [17]:
soup.select(".multi_select")[-2]

<ul class="multi_select text" id="with_genres" name="with_genres[]">
<li data-value="28"><a class="no_click" href="/discover/movie?with_genres=28">Action</a></li>
<li data-value="12"><a class="no_click" href="/discover/movie?with_genres=12">Adventure</a></li>
<li data-value="16"><a class="no_click" href="/discover/movie?with_genres=16">Animation</a></li>
<li data-value="35"><a class="no_click" href="/discover/movie?with_genres=35">Comedy</a></li>
<li data-value="80"><a class="no_click" href="/discover/movie?with_genres=80">Crime</a></li>
<li data-value="99"><a class="no_click" href="/discover/movie?with_genres=99">Documentary</a></li>
<li data-value="18"><a class="no_click" href="/discover/movie?with_genres=18">Drama</a></li>
<li data-value="10751"><a class="no_click" href="/discover/movie?with_genres=10751">Family</a></li>
<li data-value="14"><a class="no_click" href="/discover/movie?with_genres=14">Fantasy</a></li>
<li data-value="36"><a class="no_click" href="/discover/movie?with_ge

In [18]:
# Grabbing the genres of the movies
def genre_grabber(soup: BeautifulSoup) -> list:
    genres = soup.select(".multi_select")[-2]
    my_genres = []
    for gen in genres.find_all("li"): # found them in list item (li) using inspect
        a_tag = gen.find("a")
        if a_tag:
            my_genres.append(a_tag.text.strip())
    return my_genres
genre_grabber(vaccumed)
    

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [19]:
# ok so what I plan to do is that
# I already have a list of all the movie content IDs
# gonna iterate through them to create a new list object which will be my inputs for my cast func
html_content(vaccumed)
load = []
for info in html_content(vaccumed):
    load.append('https://www.themoviedb.org/' + info)
load

['https://www.themoviedb.org/movie/939243',
 'https://www.themoviedb.org/movie/124198',
 'https://www.themoviedb.org/movie/539972',
 'https://www.themoviedb.org/movie/993710',
 'https://www.themoviedb.org/movie/124928',
 'https://www.themoviedb.org/movie/426063',
 'https://www.themoviedb.org/movie/762509',
 'https://www.themoviedb.org/movie/125578',
 'https://www.themoviedb.org/movie/811941',
 'https://www.themoviedb.org/movie/558449',
 'https://www.themoviedb.org/movie/912649',
 'https://www.themoviedb.org/movie/970450',
 'https://www.themoviedb.org/movie/124132',
 'https://www.themoviedb.org/movie/141008',
 'https://www.themoviedb.org/movie/122224',
 'https://www.themoviedb.org/movie/933260',
 'https://www.themoviedb.org/movie/845781',
 'https://www.themoviedb.org/movie/426889',
 'https://www.themoviedb.org/movie/108101',
 'https://www.themoviedb.org/movie/402431']

In [23]:
# grabbing the casts of alll the movies
def cast_grabber(soup: BeautifulSoup) -> list: 

    cast_list = []

    def castisize():
        cards = soup.select("li.card")

        # this loop creates a beautifulsoup instance of every link request
        for card in cards:
              name = card.find("p").get_text(strip=True)
              print(name)
              cast_list.append(name)
        
    castisize()
    return cast_list  

# all the leetcoding makes sense now, epic use of hashmap 
all_casts = {}
for link in load:
     res = requests.get(link,headers=needed_headers)
     soup = BeautifulSoup(res.text,"lxml")
     all_casts[link] = cast_grabber(soup)



जिम कैरी
Ben Schwartz
कियानू रीव्स
इड्रिस एल्बा
Colleen O'Shaughnessey
जेम्स मार्सडेन
Tika Sumpter
Lee Majdoub
Krysten Ritter
Graciela Borges
Rita Cortese
Carolina Fal
Evangelina Salazar
Silvia Süller
Aaron Taylor-Johnson
Ariana DeBose
Fred Hechinger
Alessandro Nivola
Christopher Abbott
रसल क्रो
Yuri Kolokolnikov
Levi Miller
Tom Reed
कैमरुन डिएज़
Jamie Foxx
McKenna Roberts
Rylan Jackson
Glenn Close
Kyle Chandler
Andrew Scott
Jamie Demetriou
Fola Evans-Akingbola
Lily-Rose Depp
निकोलस हॉल्ट
Bill Skarsgård
Aaron Taylor-Johnson
विलेम डाफ़ो
एम्मा कोरिन
Ralph Ineson
Simon McBurney
Adéla Hesová
Aaron Pierre
Kelvin Harrison Jr.
Tiffany Boone
Kagiso Lediga
Preston Nyman
Blue Ivy Carter
John Kani
Mads Mikkelsen
Seth Rogen
Pat O’Brien
Dick Powell
Ginger Rogers
Allen Jenkins
Grant Mitchell
Joseph Cawthorn
Joan Wheeler
Henry O'Neill
Charles Lane
N.T. Rama Rao Jr.
Saif Ali Khan
Prakash Raj
श्रीकांत
Kalaiyarasan
Shruti Marathe
Janhvi Kapoor
Shine Tom Chacko
Ajay
Paul Mescal
Denzel Washington
पेद्रो प

In [24]:
# iteratin through the items of the dict to get clean result
for url, cast in all_casts.items():
     print(f"Cast for {url} -> {cast}")

Cast for https://www.themoviedb.org/movie/939243 -> ['जिम कैरी', 'Ben Schwartz', 'कियानू रीव्स', 'इड्रिस एल्बा', "Colleen O'Shaughnessey", 'जेम्स मार्सडेन', 'Tika Sumpter', 'Lee Majdoub', 'Krysten Ritter']
Cast for https://www.themoviedb.org/movie/124198 -> ['Graciela Borges', 'Rita Cortese', 'Carolina Fal', 'Evangelina Salazar', 'Silvia Süller']
Cast for https://www.themoviedb.org/movie/539972 -> ['Aaron Taylor-Johnson', 'Ariana DeBose', 'Fred Hechinger', 'Alessandro Nivola', 'Christopher Abbott', 'रसल क्रो', 'Yuri Kolokolnikov', 'Levi Miller', 'Tom Reed']
Cast for https://www.themoviedb.org/movie/993710 -> ['कैमरुन डिएज़', 'Jamie Foxx', 'McKenna Roberts', 'Rylan Jackson', 'Glenn Close', 'Kyle Chandler', 'Andrew Scott', 'Jamie Demetriou', 'Fola Evans-Akingbola']
Cast for https://www.themoviedb.org/movie/124928 -> []
Cast for https://www.themoviedb.org/movie/426063 -> ['Lily-Rose Depp', 'निकोलस हॉल्ट', 'Bill Skarsgård', 'Aaron Taylor-Johnson', 'विलेम डाफ़ो', 'एम्मा कोरिन', 'Ralph Ineso